In [2]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from dataset import DSpritesDataModule
from models.vae import VAE
from models.factor_vae import FactorVAE
from utils import visualize_reconstructions, save_individual_latent_traversal_grids, get_accelerator

import os

In [11]:
class Args:
    pass

args = Args()

# --- IMPOSTAZIONI MODELLO E TRAINING ---
args.model_type = 'factor_vae'

args.latent_dim = 10
args.epochs = 1
args.batch_size = 16
args.num_workers = 2
args.seed = 1234
args.patience_early_stopping = 5
args.accelerator = get_accelerator()

# --- IMPOSTAZIONI OUTPUT E VISUALIZZAZIONE ---
args.output_dir = "generated_images_final_nb"
args.n_reconstruction_images = 8
args.n_images_for_traversal_grids = 3
args.n_traversal_steps_per_dim = 7
args.traversal_range_min = -2.5
args.traversal_range_max = 2.5

# --- IPERPARAMETRI SPECIFICI PER VAE ---
if args.model_type == 'vae':
    args.lr = 1e-4
    args.beta = 4.0

# --- IPERPARAMETRI SPECIFICI PER FactorVAE ---
elif args.model_type == 'factor_vae':
    args.lr_vae = 1e-4
    args.lr_disc = 1e-4  # Come da paper [cite: 255]
    args.gamma = 35.0    # Valore da paper per dSprites [cite: 146]
    args.disc_hidden_units = 1000 # Come da paper [cite: 258]
    args.disc_layers = 6          # Come da paper [cite: 258]

print(f"Configurazione caricata per il model_type: {args.model_type}")
for k, v in vars(args).items():
    print(f"  {k}: {v}")

Configurazione caricata per il model_type: factor_vae
  model_type: factor_vae
  latent_dim: 10
  epochs: 1
  batch_size: 16
  num_workers: 2
  seed: 1234
  patience_early_stopping: 5
  accelerator: mps
  output_dir: generated_images_final_nb
  n_reconstruction_images: 8
  n_images_for_traversal_grids: 3
  n_traversal_steps_per_dim: 7
  traversal_range_min: -2.5
  traversal_range_max: 2.5
  lr_vae: 0.0001
  lr_disc: 0.0001
  gamma: 35.0
  disc_hidden_units: 1000
  disc_layers: 6


In [12]:
pl.seed_everything(args.seed)
device = torch.device(args.accelerator if args.accelerator != "cpu" else "cpu")
print(f"Seed impostato a: {args.seed}")
print(f"Utilizzo del device: {device} (Accelerator: {args.accelerator})")

Seed set to 1234


Seed impostato a: 1234
Utilizzo del device: mps (Accelerator: mps)


In [13]:
dm = DSpritesDataModule(
    batch_size=args.batch_size,
    num_workers=args.num_workers,
    data_dir="data/dsprites"
)

In [14]:
model = None
monitor_metric = None # Metrica da monitorare per checkpoint e early stopping

if args.model_type == 'vae':
    model = VAE(latent_dim=args.latent_dim, lr=args.lr, beta=args.beta)
    monitor_metric = 'val_loss'
    print(f"Modello VAE con beta={args.beta} inizializzato.")
elif args.model_type == 'factor_vae':
    model = FactorVAE(
        latent_dim=args.latent_dim,
        lr_vae=args.lr_vae,
        lr_disc=args.lr_disc,
        gamma=args.gamma,
        disc_hidden_units=args.disc_hidden_units,
        disc_layers=args.disc_layers
    )
    monitor_metric = 'val_vae_loss'
    print(f"Modello FactorVAE con gamma={args.gamma} inizializzato.")
else:
    raise ValueError("Invalid model_type specified in args.")

if model:
    print(f"Struttura del modello:\n{model}")

Modello FactorVAE con gamma=35.0 inizializzato.
Struttura del modello:
FactorVAE(
  (encoder): Encoder(
    (conv1): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (conv2): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (conv3): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (conv4): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (fc_intermediate): Linear(in_features=1024, out_features=128, bias=True)
    (fc_mean): Linear(in_features=128, out_features=10, bias=True)
    (fc_logvar): Linear(in_features=128, out_features=10, bias=True)
  )
  (decoder): Decoder(
    (fc1): Linear(in_features=10, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=1024, bias=True)
    (upconv1): ConvTranspose2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (upconv2): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (upconv3): ConvTranspose2d(32,

In [15]:
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir, exist_ok=True)

# Callback per salvare il miglior modello
checkpoint_callback = ModelCheckpoint(
    dirpath=os.path.join("checkpoints", args.model_type), # Salva in checkpoints/model_type/
    filename=f"{{epoch}}-{{{monitor_metric}:.2f}}",
    save_top_k=1,
    verbose=True,
    monitor=monitor_metric,
    mode='min'
)

# Callback per early stopping
early_stop_callback = EarlyStopping(
    monitor=monitor_metric,
    patience=args.patience_early_stopping,
    verbose=True,
    mode='min'
)

# Logger per TensorBoard
tensorboard_logger = TensorBoardLogger(
    save_dir="logs/", # Salva in logs/
    name=args.model_type
)

print("Callbacks e Logger configurati.")

Callbacks e Logger configurati.


In [16]:
trainer = pl.Trainer(
    max_epochs=args.epochs,
    accelerator=args.accelerator,
    devices=1 if args.accelerator != "cpu" else None,
    callbacks=[checkpoint_callback, early_stop_callback],
    logger=tensorboard_logger,
    enable_progress_bar=True
)
print("Trainer inizializzato.")

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Trainer inizializzato.


In [ ]:
print(f"Inizio addestramento per {args.model_type}...")
trainer.fit(model, datamodule=dm)
print("Addestramento completato.")

In [17]:
if dm.dsprites_test is None:
    print("Setup del datamodule per lo stage 'test'...")
    dm.setup(stage='test')

test_dataloader = dm.test_dataloader()

if not test_dataloader or len(test_dataloader.dataset) == 0:
    print("ATTENZIONE: Test dataloader non disponibile o vuoto.")
else:
    print(f"Test dataloader pronto con {len(test_dataloader.dataset)} campioni.")

Setup del datamodule per lo stage 'test'...
Dataset split: Train=516095, Val=110592, Test=110593
Test dataloader pronto con 110593 campioni.


In [18]:
best_model_path = "./checkpoints/factor_vae/fvae.ckpt"
trained_model_for_viz = None

if not best_model_path or not os.path.exists(best_model_path):
    print(f"Nessun modello '{args.model_type}' trovato in {best_model_path} o il percorso non esiste.")
    print("Sto usando il modello corrente in memoria (potrebbe non essere il migliore).")
    trained_model_for_viz = model # Usa il modello corrente in memoria
else:
    print(f"Caricamento del miglior modello da: {best_model_path}")
    if args.model_type == 'vae':
        trained_model_for_viz = VAE.load_from_checkpoint(best_model_path)
    elif args.model_type == 'factor_vae':
        # Nota: Se gli iperparametri passati a FactorVAE(...) non sono salvati con save_hyperparameters()
        # potresti doverli passare di nuovo qui. Ma Lightning solitamente li gestisce.
        trained_model_for_viz = FactorVAE.load_from_checkpoint(best_model_path)

if trained_model_for_viz:
    trained_model_for_viz.to(device)
    trained_model_for_viz.eval()
    print("Modello caricato e impostato in modalità evaluazione.")
else:
    print("Errore: Nessun modello disponibile per la visualizzazione.")

Caricamento del miglior modello da: ./checkpoints/factor_vae/fvae.ckpt
Modello caricato e impostato in modalità evaluazione.


In [19]:
if trained_model_for_viz and test_dataloader and len(test_dataloader.dataset) > 0:
    print(f"\nVisualizzazione e salvataggio delle ricostruzioni per {args.model_type}...")
    
    reconstruction_output_dir = os.path.join(args.output_dir, args.model_type, "simple_reconstructions")
    if not os.path.exists(reconstruction_output_dir):
        os.makedirs(reconstruction_output_dir, exist_ok=True)
        
    visualize_reconstructions(
        trained_model_for_viz,
        test_dataloader,
        n_images=args.n_reconstruction_images,
        device=device,
        output_dir=reconstruction_output_dir,
        output_filename=f"reconstructions_ep{trained_model_for_viz.current_epoch if hasattr(trained_model_for_viz, 'current_epoch') else 'N_A'}.png"
    )
    print(f"Immagini di ricostruzione salvate in: {reconstruction_output_dir}")
else:
    print("Saltata visualizzazione delle ricostruzioni (modello o test_dataloader non pronti).")

# %% [markdown]
# ### 10.3 Visualizzazione degli Attraversamenti Latenti (Latent Traversals)
# 
# Generiamo griglie di attraversamento per alcune immagini del test set. Ogni griglia mostra come l'immagine generata cambia quando si attraversa una singola dimensione latente, mantenendo le altre fisse. Questo aiuta a valutare qualitativamente il disentanglement.

# %%
if trained_model_for_viz and test_dataloader and len(test_dataloader.dataset) > 0:
    print(f"\nGenerazione e salvataggio delle griglie di attraversamento latente per {args.model_type}...")
    
    traversal_output_dir = os.path.join(args.output_dir, args.model_type, "individual_traversal_grids")
    if not os.path.exists(traversal_output_dir):
        os.makedirs(traversal_output_dir, exist_ok=True)
        
    save_individual_latent_traversal_grids(
        trained_model_for_viz,
        test_dataloader,
        n_images_to_show=args.n_images_for_traversal_grids,
        n_traversal_steps=args.n_traversal_steps_per_dim,
        traverse_range=(args.traversal_range_min, args.traversal_range_max),
        device=device,
        output_dir=traversal_output_dir,
        filename_prefix=f"traversal_grid_ep{trained_model_for_viz.current_epoch if hasattr(trained_model_for_viz, 'current_epoch') else 'N_A'}_img_"
    )
    print(f"Griglie di attraversamento salvate in: {traversal_output_dir}")
else:
    print("Saltata generazione delle griglie di attraversamento (modello o test_dataloader non pronti).")



Visualizzazione e salvataggio delle ricostruzioni per factor_vae...


/Users/carlodenardin/miniconda3/envs/vae-compare/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Immagine delle ricostruzioni salvata in: generated_images_final_nb/factor_vae/simple_reconstructions/reconstructions_ep0.png
Immagini di ricostruzione salvate in: generated_images_final_nb/factor_vae/simple_reconstructions

Generazione e salvataggio delle griglie di attraversamento latente per factor_vae...
Griglia di traversata per immagine 1 salvata in: generated_images_final_nb/factor_vae/individual_traversal_grids/traversal_grid_ep0_img_1.png
Griglia di traversata per immagine 2 salvata in: generated_images_final_nb/factor_vae/individual_traversal_grids/traversal_grid_ep0_img_2.png
Griglia di traversata per immagine 3 salvata in: generated_images_final_nb/factor_vae/individual_traversal_grids/traversal_grid_ep0_img_3.png
Griglie di attraversamento salvate in: generated_images_final_nb/factor_vae/individual_traversal_grids
